## Ajoke Yusuf MicroTask Contribution

### Table of Content
- Introduction
- Dataset Overview
- Data Cleaning
- Data Visualization using **Bokeh plot**
- Interactive Bokeh plot

### Introduction:

The main aim of this project is to carry out an exploratory data analysis with a subset of the dataset using **Bokeh plots** for visualization. In this project, the link to download the dataset was provided. The pdf containing the metadata can be found in :
 https://www.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_green.pdf

In [ ]:
pip install pyarrow

In [ ]:
#Importing necessary libraries

import pandas as pd 
import glob #to match file pattern of .parquet

#To arrange multiple plots in a grid
from bokeh.layouts import layout, gridplot

from bokeh.models import CustomJS, ColumnDataSource, HoverTool, TapTool

from bokeh.io import curdoc

#To create and display a plot in the notebook
from bokeh.plotting import figure, show

#To configure the notebook to display Bokeh plots inline
from bokeh.io import output_notebook

In [ ]:
output_notebook()

### Dataset Overview
The dataset link was provided and downloaded from https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page. The dataset downloaded was **2022 Green Taxi Trip Record** in parquet format which span from January to November. The datset consist of 2137732 records and 23 variable in the dataframe. See the data dictionary(meta data) to understand the dataset variables

In [ ]:
# Path to the directory containing the parquet files
path = r'C:\Users\user\Downloads'

# Get a list of all the parquet files in the directory
parquet_files = glob.glob(path + '/*.parquet')

# Read each parquet file into a separate pandas DataFrame
data_frames = [pd.read_parquet(file) for file in parquet_files]

# Combine the separate data frames into a single data frame
combined_df = pd.concat(data_frames)
combined_df.head()


In [ ]:
#Returns the number of rows and columns in the dataframe
print("There are {} records in the dataframe".format(combined_df.shape[0]))
print("There are {} fields in the dataframe".format(combined_df.shape[1]))     

In [ ]:
#summary statistics of the dataset
combined_df.describe()

### Data Cleaning

In [ ]:
#percentage of null values found in all the columns
combined_df.isnull().sum()*100/len(combined_df)

In [ ]:
combined_df.info()

In [ ]:
#Dropping 'ehial_fee' column becuase it contains 100% null values
combined_df.drop(columns=['ehail_fee','airport_fee'],inplace=True)

In [ ]:
#Dropping of null values
combined_df.dropna(subset=['store_and_fwd_flag','RatecodeID','passenger_count','payment_type','trip_type',
                           'congestion_surcharge'],inplace=True)

In [ ]:
combined_df.columns

In [ ]:
combined_df.info()

In [ ]:
#checking for duplicate values
combined_df.duplicated().sum()

In [ ]:
combined_df['RatecodeID'].unique()

In [ ]:
#A dictionary that maps the rate codes to their correspondong description from the metadata
rate_code= {1.0: 'Standard rate', 2.0: 'JFK', 3.0: 'Newark', 4.0: 'Nassau or Westchester', 
            5.0: 'Negotiated fare', 6.0: 'Group ride'}

# replace the rate codes in the dataframe with their descriptions
combined_df['RatecodeID'] = combined_df['RatecodeID'].replace(rate_code)

#Checking if the replacement has been made
combined_df['RatecodeID']

In [ ]:
combined_df['payment_type'].unique()

In [ ]:
#create a dictinary that maps the payment type code to its correspondong description from the metadata
payment_code ={1: 'Credit card', 2: 'Cash', 3: 'No charge', 4: 'Dispute', 5: 'Negotiated fare'}

# replace the rate codes in the dataframe with their descriptions
combined_df['payment_type'] = combined_df['payment_type'].replace(payment_code)

#Checking if the replacement has been made
combined_df['payment_type']

In [ ]:
#create a dictinary that maps the trip-type to its correspondong description from the metadata
trip_type = {1: 'Street-hail', 2: 'Discharge'}

# replace the rate codes in the dataframe with their descriptions
combined_df['trip_type'] = combined_df['trip_type'].replace(trip_type)

#Checking if the replacement has been made
combined_df['trip_type']

In [ ]:
# convert the dtypes of "lpep_pickup_datetime" column to datetime
combined_df['lpep_pickup_datetime']= pd.to_datetime(combined_df['lpep_pickup_datetime'])


# extract the month, day, and time components into separate columns
combined_df['pickup_month']= combined_df['lpep_pickup_datetime'].dt.strftime('%b')
combined_df['pickup_day']= combined_df['lpep_pickup_datetime'].dt.strftime('%A')
combined_df['pickup_time(hours)']= combined_df['lpep_pickup_datetime'].dt.strftime('%H')

In [ ]:
combined_df

In [ ]:
# convert the dtypes of "lpep_dropoff_datetime" column to datetime
combined_df['lpep_dropoff_datetime']= pd.to_datetime(combined_df['lpep_dropoff_datetime'])


# extract the month, day, and time components into separate columns
combined_df['dropoff_month']= combined_df['lpep_dropoff_datetime'].dt.strftime('%b')
combined_df['dropoff_day']= combined_df['lpep_dropoff_datetime'].dt.strftime('%d')
combined_df['dropoff_time']= combined_df['lpep_dropoff_datetime'].dt.strftime('%H')

In [ ]:
#drop of the "lpep_pickup_datetime" and "lpep_dropoff_datetime" columns since the day,month and time has been extracted
combined_df.drop(columns=['lpep_pickup_datetime','lpep_dropoff_datetime'], inplace=True)

In [ ]:
combined_df

- Available theme
caliber, dark_minimal, light_minimal, night_sky, contrast

### ANALYSIZING & DATA VISUALIZATION USING VARIOUS PLOT IN **BOKEH**

In this section, I will be be investigating and questioning the dataset to get insight and visualize the insight with various **bokeh plot**

### Question 1

Which payment type was widely used by passengers?

In [ ]:
combined_df.groupby('payment_type')['payment_type'].count()

### Visualization: Bar plot

In [ ]:
from bokeh.models import NumeralTickFormatter #to format axis

#create a column data source
source = ColumnDataSource(data=dict(payment_types=combined_df['payment_type'].unique(), 
                                     values=combined_df.groupby('payment_type')['payment_type'].count()))

#create a figure object and set the background color to grey
p = figure(x_range=source.data['payment_types'], plot_height=600, title="Payment Type Frequency",
           background_fill_color='darkgrey')

#add a vbar(vertical bar) glyph
p.vbar(x='payment_types', top='values', width=0.9, source=source,color='firebrick') 
      
#labelling of x axis
p.xaxis.axis_label = "Payment Type"

#labelling of y axis
p.yaxis.axis_label = "Total Payment Amount"
p.xaxis.major_label_orientation = 0.5

p.grid.visible = True
#p.xticks_label_text_font_size ='14pt'

#format the Y-axis ticks as whole number
p.yaxis.formatter = NumeralTickFormatter(format='0,0')
show(p)


### [Click TO View Plot]('https://github.com/Ajoke23/Ajoke_Outreachy23_BokehContribution/blob/main/Plot/Barplot.PNG')

### Observation

-  **Credit card** is the most widely used by passenegrs for payment.

### Question 2

 Is there any correlation between the time of pickup and total amount?

### Visualization: Scatterplot

In [ ]:
from bokeh.layouts import column
from bokeh.transform import jitter
from bokeh.plotting import figure, show

# Create a ColumnDataSource object
source = ColumnDataSource(combined_df)

# Create the first figure object
p1 = figure(title="Pickup time vs Total Amount", x_axis_label="Pickup Time(hours)",
            y_axis_label="Total Amount",width=700,height=400,background_fill_color='lightgrey')

# create a first scatter subplot
p1.scatter(x="pickup_time(hours)", y="total_amount", source=source, size=10, color='darkmagenta', alpha=0.4)

#set grid line color to none
p1.xgrid.grid_line_color = None

# Create a second figure subplot
p2 = figure(title="Pickup time vs Fare Amount", x_axis_label="Pickup Time(Hours)",
            y_axis_label="Fare Amount", width=700, height=400,background_fill_color='lightgrey')

# Add a scatter glyph for p2
p2.scatter(x="pickup_time(hours)", y="fare_amount", source=source, size=10, color='goldenrod', alpha=0.4)

#set grid line color to none
p2.xgrid.grid_line_color = None

# Show the plot
show(column(p1,p2))

### [Click To View Plot]('https://github.com/Ajoke23/Ajoke_Outreachy23_BokehContribution/blob/main/Plot/Scatterplot1.png')

### Observation

- The first subplot shows the relationship between pickup time (in hours) and the total premium amount paid and from the plot there exist no correlation. The second subplot shows the relationship between pickup time (in hours) and the fare amount and we can also deduce that there exist no correlation between pickup_time and fare amount

- There seems to be a cluster of data points and also a few outliers in both plots, indicating some trips with unusually high total amount/fare amount paid by the passenger irrespective to their pickup time which implies that it cannot be used to optimized price

- This analysis further deduced that the hours of the day in which a passenger took a trip fare is independent on the fare amount and total amount paid


### Question 3

Could the type of RateCodeID be a dermining factor of Fare Amount?

### Visualization: Whisker plot

In [ ]:
from bokeh.models import ColumnDataSource, Whisker
from bokeh.plotting import figure, show
from bokeh.transform import factor_cmap, jitter

classes = list(sorted(combined_df["RatecodeID"].unique()))

#to create figure object
p = figure(height=400, x_range=classes, background_fill_color="darkgrey",
           title="Final Rate Code vs Fare Amount")
p.xgrid.grid_line_color = None

g = combined_df.groupby("RatecodeID")
upper = g.fare_amount.quantile(0.80)
lower = g.fare_amount.quantile(0.20)
source = ColumnDataSource(data=dict(base=classes, upper=upper, lower=lower))

error = Whisker(base="base", upper="upper", lower="lower", source=source,
                level="annotation", line_width=0.5)
error.upper_head.size=8
error.lower_head.size=8
p.add_layout(error)

#add a circle glyph
p.circle(jitter("RatecodeID", 0.3, range=p.x_range), "fare_amount", source=combined_df,
         alpha=0.5, size=13, line_color="white",
         color=factor_cmap("RatecodeID", "Spectral6", classes))

#to set theme color
curdoc().theme= 'dark_minimal'


p.xaxis.major_label_orientation = 0.5

#show the plot
show(p)

### [Click To View Plot](https://github.com/Ajoke23/Ajoke_Outreachy23_BokehContribution/blob/main/Plot/WhiskerPlot.PNG)

### Observation

- The whisker plot shows the distribution of various RateCodeID categories across price of fare. The boxes represent the interquartile range (IQR), with the lower and upper whiskers.
- Newark, on the other hand, has the lowest median fare amount and the narrowest range of fares.
- Overall, the plot suggests that the RateCodeID category is a significant predictor of fare amount.

### Question 4

Which is the most populous trip type?

### Visualization: Pie plot

In [ ]:
from bokeh.palettes import Category10 #select catergory10 palette
from bokeh.plotting import figure, show
from bokeh.transform import cumsum
from bokeh.io import curdoc
from math import pi


# Create some sample data
data = combined_df['trip_type'].value_counts()
df = pd.DataFrame({'trip_type': data.index, 'count': data.values})

# Calculate the percentage
df['angle'] = df['count']/df['count'].sum()* 2*pi
df['color'] = Category10[10][:len(df)]
df['percentage'] = df['count']/df['count'].sum()*100

# Create a figure object
p = figure(title="Most Populoous Trip type", toolbar_location=None, tools="hover", 
           tooltips=[("Trip Type","@trip_type"),("Percentage", "@percentage{0.2f}%")])

# Add a wedge glyph
p.wedge(x=0, y=1, radius=0.4, start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='trip_type', source=df)


# Set the legend location
p.legend.location = "top_right"
p.axis.visible = False
p.grid.grid_line_color = None

#to set theme color
curdoc().theme= 'caliber'
# Show the plot
show(p)


### [Click To View Plot](https://github.com/Ajoke23/Ajoke_Outreachy23_BokehContribution/blob/main/Plot/Pie%20Chart1.png) 

### Observation

- From the pie chart,'street-hail' happens to be the most populous type of trip having a percentage of 95.16%

### Question 5

What are the top 2 day with the most recorded trip?

### Visualization: Verical Bar Plot

In [ ]:
from bokeh.palettes import Category20
from bokeh.io import curdoc #to apply theme

# Get the data
data = combined_df['pickup_day'].value_counts()

# Create a ColumnDataSource object
source = ColumnDataSource(data=dict(days=data.index.tolist(), counts=data.values.tolist()))

# Create a figure object
p = figure(title="Pickup Day Frequency", y_range=data.index.tolist(), x_range=(0, data.max()*1.1))

# Add a vbar glyph
p.hbar(y='days', right='counts', height=0.8, color=Category20[10][4], source=source)

#set theme to dark
curdoc().theme= 'dark_minimal'

# Set the axis labels
p.yaxis.axis_label = "Day"
p.xaxis.axis_label = "Frequency"

#format the Y-axis ticks as whole number
p.xaxis.formatter = NumeralTickFormatter(format='0,0')

# Show the plot
show(p)


### [Click To View Plot](https://github.com/Ajoke23/Ajoke_Outreachy23_BokehContribution/blob/main/Plot/Vertical_Plot.PNG)

### Observation

- Thursday and Friday recorded the most trips for the week which implies more passenger took taxi at both Thursday and Friday.
- From the barchart, we could also observe that weekends(Saturday and Sunday) recorded the least trip for the week. 
- Most passengers have more outing during the weekdays than weekends

### Question 6

Which day has the highest congestion surchage?

In [ ]:
combined_df.groupby('pickup_day')[['congestion_surcharge','tolls_amount']].sum().sort_values(by='congestion_surcharge')

### Visualization: Double Plot

In [ ]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.palettes import Category10
from bokeh.io import curdoc
# groupby the pickup day and sum the fare and congestion surcharge
grouped_df = combined_df.groupby('pickup_day')[['congestion_surcharge', 'tolls_amount']].sum()

# Create a ColumnDataSource object
source = ColumnDataSource(grouped_df)


# Create a figure object
p = figure(title="Fare Amount and Congestion Surcharge by Pickup Day", x_range=grouped_df.index.tolist(),
 
           y_axis_label="Fare Amount ($)", y_range=(0, grouped_df['congestion_surcharge'].max()*1.1))
# Add the bar glyph for fare amount
p.vbar(x='pickup_day', top='congestion_surcharge', source=source, width=0.7, color='darkgrey', 
       legend_label='congestion_surcharge')

#p.vbar(x='pickup_day', top='tolls_amount', source=source, width=0.7, color='goldenrod')

# Add the line glyph for congestion surcharge
p.line(x='pickup_day', y='tolls_amount', source=source, color='green', line_width=3, legend_label='tolls_amount')

# Set the legend location
p.legend.location = "top_left"

#set a theme
curdoc().theme= 'night_sky'

#format the Y-axis ticks as whole number
p.yaxis.formatter = NumeralTickFormatter(format='0,0')

# Show the plot
show(p)


### [Click To View Plot](https://github.com/Ajoke23/Ajoke_Outreachy23_BokehContribution/blob/main/Plot/DoublePlot.png)

### Observation

- Week days has the highest congestion surcharge and tolls amount paid.
- There eist a direct proportionality between congestion surcharge and tolls amount which implies the higher the congestion charge the higher the tolls amount 

### Question 7

What pairwise relationship exist between Tolls Amount, MTA Tax and Tip Amount?

### Visualization: Subplot

In [ ]:
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.layouts import gridplot

# Create a ColumnDataSource object
source = ColumnDataSource(combined_df)

# Define the tools to include in the plot
tools = "box_select,lasso_select,reset,help,save"

# Define the scatterplot matrix
s1 = figure(width=300, height=300, tools=tools, title='Tolls Amount vs MTA Tax')
s1.circle('tolls_amount', 'mta_tax', source=source)

s2 = figure(width=300, height=300, tools=tools, title='Tolls Amount vs Tip Amount')
s2.circle('tolls_amount', 'tip_amount', source=source)

s3 = figure(width=300, height=300, tools=tools, title='MTA Tax vs Tip Amount')
s3.circle('mta_tax', 'tip_amount', source=source)

# Put all the plots in a grid layout
grid = gridplot([[s1, s2,s3]])

curdoc().theme= 'contrast'

# Show the plot
show(grid)


### [Click To View Plot](https://github.com/Ajoke23/Ajoke_Outreachy23_BokehContribution/blob/main/Plot/Gridplot.PNG)

### Observation

- There seems to be a weak positive correlation between Tolls amount and tip amount.
- Outliers are founds in all the subplot which need to be further investigated to know if it was an error of recording or a ridiculous hike in price

### Bokeh Subplot

In [ ]:
from bokeh.layouts import row
from bokeh.models import NumeralTickFormatter #to format axis
from bokeh.io import curdoc

amount_spent = combined_df.groupby('passenger_count')[['tip_amount']].sum()

p1 = figure(width=350, height=300, background_fill_color='black')
p1.square(x=amount_spent.index, y=amount_spent['tip_amount'], size=13, color='firebrick', alpha=0.7)

p2 = figure(width=350, height=300, background_fill_color='darkgrey')
p2.circle(x=amount_spent.index, y=amount_spent['tip_amount'], size=13, color='goldenrod', alpha=0.6)

p3 = figure(width=350, height=300, background_fill_color='darkcyan')
p3.plus(x=amount_spent.index, y=amount_spent['tip_amount'], size=13, color='crimson', alpha=0.4)

#to make the gridline visible
p.grid.visible = True

curdoc().theme= 'light_minimal'

#format the Y-axis ticks as whole number
p.yaxis.formatter = NumeralTickFormatter(format='0,0')

show(row(p1,p2,p3))

### [Click To View Plot](https://github.com/Ajoke23/Ajoke_Outreachy23_BokehContribution/blob/main/Plot/Subplot.PNG)

### INTERACTIVE BOKEH PLOT

### Data Table Plot

In [ ]:
from bokeh.layouts import column
from bokeh.models import (ColumnDataSource, DataTable, HoverTool, IntEditor,
                          NumberEditor, NumberFormatter, SelectEditor,
                          StringEditor, StringFormatter, TableColumn)

#select from subset of data from index 1 to 200
df_selected = combined_df.iloc[2:201]


source = ColumnDataSource(df_selected)

flags = sorted(df_selected["store_and_fwd_flag"].unique())
codeid = sorted(df_selected["RatecodeID"].unique())
payment = sorted(df_selected["payment_type"].unique())
#pickup_days = sorted(combined_df["pickup_day"].unique())
dropoff_months= sorted(df_selected["dropoff_month"].unique())

columns = [
    TableColumn(field="store_and_fwd_flag", title="Flag",
                editor=SelectEditor(options=flags),
                formatter=StringFormatter(font_style="bold")),
    TableColumn(field="RatecodeID", title="RateCode",
                editor=StringEditor(completions=codeid)),
    TableColumn(field="payment_type", title="Payment_Type",
                editor=StringEditor(completions=payment)),
    TableColumn(field="dropoff_month", title="Month(dropoff)", editor=SelectEditor(options=dropoff_months)),
    TableColumn(field="trip_type", title="Trip_Type",editor=IntEditor()),
    TableColumn(field="pickup_time(hours)", title="Hours(pickup)", editor=IntEditor()),
    TableColumn(field="PULocationID", title="Location_ID", editor=IntEditor())
]
data_table = DataTable(source=source, columns=columns, editable=True, width=800,
                       index_position=-1, index_header="row index", index_width=60)

#create a figure object
p = figure(width=800, height=400, tools="pan,wheel_zoom,xbox_select,reset,save", active_drag="xbox_select")

pth = p.circle(x="index", y="pickup_time(hours)", fill_color="#396285", size=8, alpha=0.5, source=source)
dot = p.circle(x="index", y="PULocationID", fill_color="#CE603D", size=8, alpha=0.5, source=source)

tooltips = [
    ("Flag", "@store_and_fwd_flag"),
    ("RateCode", "@RatecodeID"),
    ("Payment_Type", "@payment_type"),
    #("Passenger", "@passenger_count"),
    #("P_Day", "@pickup_day"),
    ("Month(dropoff)", "@dropoff_month"),
    ("Trip_Type", "@trip_type"),
    #("Class", "@class"),
]
pth_hover_tool = HoverTool(renderers=[pth], tooltips=tooltips + [("Pickup Hours", "@pickup_time(hours)")])
pu_hover_tool = HoverTool(renderers=[dot], tooltips=tooltips + [("PULocationID", "@PULocationID")])

p.add_tools(pth_hover_tool, pu_hover_tool)

#show plot
show(column(p, data_table))

### [Click To View Plot](https://github.com/Ajoke23/Ajoke_Outreachy23_BokehContribution/blob/main/Plot/Data%20Table%20Plot.PNG)